# Calculate Embedding Similarity Score


In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/RecSys2024/
base_path = '/content/drive/MyDrive/RecSys2024/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Exp_Recsys2024_LGBM_train.ipynb  output				     Recsys2024_ensemble.ipynb
Exp_Recsys2024_preprocess.ipynb  preprocess2_embed_similarity.ipynb  Recsys2024_LGBM_test.ipynb
feature_output			 preprocess2.ipynb		     Recsys2024_LGBM_train.ipynb
input				 preprocess_article.ipynb	     Recsys2024_preprocess.ipynb
models				 preprocess_create_embed.ipynb
old_Recsys2024_LGBM_train.ipynb  Recsys2024_EDA.ipynb


In [2]:
import pandas as pd
!pip install polars
import polars as pl
import numpy as np
import pickle
import gc
from sklearn.decomposition import PCA

In [3]:
#DEBUG_MODE = True
DEBUG_MODE = False
#yyama_flag = False
yyama_flag = True
#DATA_TYPE = 'train'
#DATA_TYPE = 'valid'
DATA_TYPE = 'test'

Nhist = 5
#Nhist = 10
#dim = 32
dim = 16
#dim = 8
#dim = 4
#dim = 2

#name = f'FacebookAI/xlm-roberta-base_pca{dim}'
name = f'title_vector_pca{dim}'

input_path = base_path + 'feature_output'


In [4]:
def datetime_to_unix(df, col_name):
    return df.with_columns(
        pl.col(col_name).map_elements(lambda lst: [int(dt.timestamp()) for dt in lst])
    )

In [5]:
if yyama_flag:
    embed = pl.read_parquet(f"{input_path}/embed_pca{dim}.parquet")
    embed_yyama = pl.read_parquet(f"{input_path}/yyama_embed_pca{dim}.parquet")
    embed = embed.join(embed_yyama, on='article_id', how='inner')
    del embed_yyama
    gc.collect()
else:
    embed = pl.read_parquet(f"{input_path}/embed_pca{dim}.parquet")


In [6]:
embed.head()

article_id,contrastive_vector_pca16,image_embedding_pca16,document_vector_pca16,FacebookAI/xlm-roberta-base_pca16,google-bert/bert-base-multilingual-cased_pca16,sub_vector_pca16,title_vector_pca16,title_sub_vector_pca16
i32,list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32]
3000022,"[-0.060735, 0.027431, … -0.071681]","[0.028038, -0.477603, … 0.016473]","[0.104463, 0.23517, … 0.009239]","[-0.113583, -0.072844, … 0.005158]","[2.825378, -0.186921, … -0.037083]","[-0.108365, 0.123932, … -0.013907]","[-0.136341, -0.090135, … -0.047948]","[0.19828, 0.0197, … -0.065229]"
3000063,"[-0.281957, 0.45565, … 0.02378]","[0.579085, 0.364608, … 0.016231]","[-0.072474, 0.113448, … -0.014435]","[-0.296138, 0.033279, … -0.096418]","[-0.659043, 0.098627, … -0.093011]","[0.057012, 0.030963, … 0.094973]","[-0.181426, -0.175537, … -0.089643]","[-0.046479, -0.044283, … 0.061443]"
3000613,"[0.242772, -0.095149, … -0.059613]","[0.207974, 0.503462, … -0.004376]","[-0.033802, 0.027114, … -0.044272]","[0.3795, 0.048967, … -0.027122]","[3.244764, -0.073616, … 0.576504]","[-0.125592, -0.15548, … 0.005403]","[-0.078775, -0.070515, … -0.068617]","[-0.065991, -0.156383, … -0.148105]"
3000700,"[0.515538, 0.176318, … 0.010886]","[0.283855, -0.459856, … 0.001254]","[-0.081785, 0.163194, … -0.06585]","[-0.208387, 0.130843, … -0.130361]","[0.918782, -0.341206, … 0.005484]","[-0.057773, 0.034485, … -0.050501]","[-0.170705, -0.036604, … -0.104798]","[-0.030511, 0.236575, … 0.010521]"
3000840,"[-0.024885, 0.430401, … -0.070195]","[0.65174, -0.116109, … 0.008914]","[-0.130207, 0.187807, … -0.000638]","[0.025784, 0.107481, … 0.047712]","[2.611521, -0.390219, … -0.108079]","[-0.13284, 0.053988, … 0.142838]","[-0.137563, -0.171242, … -0.039644]","[0.095699, -0.014137, … -0.11007]"


In [7]:
embed = embed.select('article_id', name)

In [8]:
embed.head()

article_id,title_vector_pca16
i32,list[f32]
3000022,"[-0.136341, -0.090135, … -0.047948]"
3000063,"[-0.181426, -0.175537, … -0.089643]"
3000613,"[-0.078775, -0.070515, … -0.068617]"
3000700,"[-0.170705, -0.036604, … -0.104798]"
3000840,"[-0.137563, -0.171242, … -0.039644]"


In [9]:
if DEBUG_MODE:
    impression_article = pl.read_parquet(f"{input_path}/small_{DATA_TYPE}_impression_article.parquet")
else:
    impression_article = pl.read_parquet(f"{input_path}/{DATA_TYPE}_impression_article.parquet")

In [10]:
impression_article = impression_article.join(embed,how='left', left_on='article_id_inview', right_on='article_id')
impression_article.head()

<ipython-input-10-334cf75fc963>:1: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  impression_article = impression_article.join(embed,how='left', left_on='article_id_inview', right_on='article_id')


impression_id,user_id,article_id_inview,title_vector_pca16
u32,u32,i32,list[f32]
6451339,35982,9796527,"[0.204034, 0.056298, … -0.067383]"
6451339,35982,7851321,"[0.123099, 0.156605, … -0.05194]"
6451339,35982,9798805,"[0.073464, 0.070003, … 0.110244]"
6451339,35982,9795150,"[0.028751, 0.192671, … -0.032369]"
6451339,35982,9531110,"[-0.210194, -0.115664, … 0.024252]"


In [11]:
if DEBUG_MODE:
  input_path = base_path + 'input/ebnerd_small/'
else:
  input_path = base_path + 'input/ebnerd_large/'

if DATA_TYPE == 'train':
    df_user = pl.read_parquet(input_path + 'train/history.parquet')
elif DATA_TYPE == 'valid':
    df_user = pl.read_parquet(input_path +  'validation/history.parquet')
else:
    input_path = base_path + 'input/ebnerd_testset/'
    df_user = pl.read_parquet(input_path + 'test/history.parquet')

In [12]:
df_user = df_user.select(['user_id', 'article_id_fixed'])
df_user.head()

user_id,article_id_fixed
u32,list[i32]
40107,"[9676294, 9763942, … 9776147]"
40254,"[9759284, 9759389, … 9789473]"
40926,"[9759355, 9759707, … 9789896]"
40963,"[9759929, 9757526, … 9789475]"
41474,"[9762373, 9761561, … 9780406]"


In [13]:
df_user = df_user.with_columns(
    pl.col("article_id_fixed").list.tail(Nhist).alias(f"last_{Nhist}_articles"),
)

In [14]:
df_user.head(2)

user_id,article_id_fixed,last_5_articles
u32,list[i32],list[i32]
40107,"[9676294, 9763942, … 9776147]","[9787656, 9787436, … 9776147]"
40254,"[9759284, 9759389, … 9789473]","[9788576, 9789539, … 9789473]"


In [15]:
col = f'last_{Nhist}_articles'
df_user = df_user.select(['user_id', col]).explode(col).rename({col: "article_id"})
df_user = df_user.join(embed, on='article_id', how='inner')

In [16]:
df_user.head()

user_id,article_id,title_vector_pca16
u32,i32,list[f32]
40107,9787656,"[0.381483, -0.20255, … -0.046377]"
40107,9787436,"[0.122869, -0.003064, … 0.1013]"
40107,9787501,"[-0.142153, -0.100519, … 0.049106]"
40107,9789473,"[0.327838, -0.076397, … -0.02822]"
40107,9776147,"[0.179715, 0.072929, … 0.035946]"


In [17]:
def sum_and_normalize_numpy(embeddings):
    # 全体にnp.array()を適用するとバグるので要素ごと
    embeddings = [np.array(i) for i in embeddings]
    summed_embedding = np.sum(embeddings, axis=0)
    norm = np.linalg.norm(summed_embedding)
    normalized_embedding = summed_embedding / norm
    return normalized_embedding.astype(np.float32).tolist()

df_user = df_user.group_by("user_id").agg([
    pl.col(name).map_elements(lambda x: sum_and_normalize_numpy(x)).alias("user_embedding")
])


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [18]:
impression_article = impression_article.join(df_user, on='user_id', how='left')

<ipython-input-18-d54587ab8566>:1: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  impression_article = impression_article.join(df_user, on='user_id', how='left')


In [ ]:
def cosine_similarity(v1, v2):
    v1 = np.array(v1, dtype=np.float32)
    v2 = np.array(v2, dtype=np.float32)
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return float(similarity)

col1 = name
out_name = col1.split('/')[-1] # /を含む場合はそれ以前を削除
col2 = 'user_embedding'
impression_article = impression_article.with_columns(
    pl.struct([col1, col2]).map_elements(lambda x: cosine_similarity(x[col1], x[col2])).cast(pl.Float32).alias(f"sim_{out_name}_{Nhist}hist")
)
impression_article = impression_article.drop('user_embedding', col1)

<ipython-input-19-bd77b698164c>:13: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  impression_article = impression_article.with_columns(


In [ ]:
print(impression_article.shape)
impression_article.head(3)

# Save

In [ ]:
out_path = base_path + 'feature_output'

In [22]:
# save
if DEBUG_MODE:
    impression_article.write_parquet(f"{out_path}/small_{DATA_TYPE}_impression_article_{out_name}.parquet")
else:
    impression_article.write_parquet(f"{out_path}/{DATA_TYPE}_impression_article_{out_name}.parquet")
del impression_article
gc.collect()

0

# Merge Results

In [23]:
MERGE_FLAG = False
#MERGE_FLAG = True

In [25]:
if MERGE_FLAG:
    name1 = f'xlm-roberta-base_pca{dim}'
    name2 = f'title_vector_pca{dim}'
    if DEBUG_MODE:
        df1 = pl.read_parquet(f"{out_path}/small_{DATA_TYPE}_impression_article_{name1}.parquet")
        df2 = pl.read_parquet(f"{out_path}/small_{DATA_TYPE}_impression_article_{name2}.parquet")
        if DATA_TYPE in ('train', 'valid'):
            df = df1.join(df2, on=['impression_id', 'user_id', 'article_id_inview', 'clicked'], how='inner')
        else:
            df = df1.join(df2, on=['impression_id', 'user_id', 'article_id_inview'], how='inner')
        df.write_parquet(f"{out_path}/small_{DATA_TYPE}_impression_article_w_embed.parquet")
    else:
        df1 = pl.read_parquet(f"{out_path}/{DATA_TYPE}_impression_article_{name1}.parquet")
        df2 = pl.read_parquet(f"{out_path}/{DATA_TYPE}_impression_article_{name2}.parquet")
        if DATA_TYPE in ('train', 'valid'):
            df = df1.join(df2, on=['impression_id', 'user_id', 'article_id_inview', 'clicked'], how='inner')
        else:
            df = df1.join(df2, on=['impression_id', 'user_id', 'article_id_inview'], how='inner')
        df.write_parquet(f"{out_path}/{DATA_TYPE}_impression_article_w_embed.parquet")

In [26]:
df

impression_id,user_id,article_id_inview,sim_xlm-roberta-base_pca16_5hist,sim_title_vector_pca16_5hist
u32,u32,i32,f32,f32
6451339,35982,9796527,-0.407535,0.220448
6451339,35982,7851321,0.020333,0.029117
6451339,35982,9798805,0.248214,-0.062174
6451339,35982,9795150,-0.350247,-0.038875
6451339,35982,9531110,-0.250325,-0.257405
…,…,…,…,…
0,1225161,9792362,-0.517027,-0.195047
0,1225161,9788041,-0.658522,-0.170149
0,1225161,9790135,-0.18739,-0.517657
